concrete crack detection using deep neural network


 

In [4]:
#MODEL  

#IMPORTING required libraries

from google.colab import drive
drive.mount('/content/drive')
import os
import numpy as np
import pandas as pd
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2

# Set path to input data
drive_path = '/content/drive/MyDrive/MLASSIGNMENT2'
train_path = os.path.join(drive_path, 'train')
test_path = os.path.join(drive_path, 'test')

# Set up data augmentation parameters
datagen = ImageDataGenerator(rescale=1./255)

# Load training data using the generator
train_generator = datagen.flow_from_directory(train_path, target_size=(224,224), batch_size=256, class_mode='binary')

#The weights of pre-trained MobileNetV2 model are used.

# Loading the pre-trained MobileNetV2 model
mobilenetv2 = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freezing the layers in the pre-trained model
for layer in mobilenetv2.layers:
    layer.trainable = False

# Defining the model architecture
model = Sequential()

model.add(mobilenetv2)

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))


model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

# Compiling the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Training the model
model.fit(train_generator, epochs=35)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 14986 images belonging to 2 classes.
Epoch 1/35
59/59 [==============================] - 65s 1s/step - loss: 1.4187 - accuracy: 0.5302
Epoch 2/35
59/59 [==============================] - 60s 1s/step - loss: 0.6730 - accuracy: 0.5906
Epoch 3/35
59/59 [==============================] - 60s 1s/step - loss: 0.6202 - accuracy: 0.6583
Epoch 4/35
59/59 [==============================] - 60s 1s/step - loss: 0.5785 - accuracy: 0.7060
Epoch 5/35
59/59 [==============================] - 60s 1s/step - loss: 0.5608 - accuracy: 0.7116
Epoch 6/35
59/59 [==============================] - 59s 1s/step - loss: 0.5505 - accuracy: 0.7275
Epoch 7/35
59/59 [==============================] - 60s 1s/step - loss: 0.5241 - accuracy: 0.7506
Epoch 8/35
59/59 [==============================] - 58s 981ms/step - loss: 0.5152 - accuracy: 0.7551
Epoch 9/35
59/59 [=======================

In [5]:
#Loading the test data
test_generator = datagen.flow_from_directory(
    test_path,
    target_size=(224,224),
    batch_size=1,
    class_mode=None,
    shuffle=False)

# Making predictions on the test data
filenames = test_generator.filenames
nb_samples = len(filenames)

predict = model.predict(test_generator)#, steps=np.ceil(nb_samples))

# Converting the predicted probabilities to class labels
class_labels = []
for p in predict:
    if p < 0.5:
        class_labels.append('cracked')
    else:
        class_labels.append('uncracked')
filenames2=[]
for file in filenames:
  filenames2.append(file.split('/')[1])

# Creating a DataFrame with the filenames and class labels
results = pd.DataFrame({'filename': filenames2, 'class': class_labels})
os.chdir('/content/drive/MyDrive/MLASSIGNMENT2')

# Saving the results to a CSV file
results.to_csv('results_ep35.csv', index=False)







Found 2000 images belonging to 1 classes.
2000/2000 [==============================] - 22s 11ms/step
